<a href="https://colab.research.google.com/github/bill-unitech2021/bias_detection_system/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.6 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForTokenClassification

In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


### Inizialize parameters, tokenizer, and model

In [18]:
MAX_LEN = 128
MAX_GRAD_NORM = 10

# temp_labels = ['M', 'F', 'O']
# label2id = {k: v for v, k in enumerate(temp_labels)}
# id2label = {v: k for v, k in enumerate(temp_labels)}
temp_labels = ['O', 'Generic she', 'Generic he',
               'Behavioural Stereotypes', 'i-Behavioural Stereotypes',
               'Societal Stereotypes', 'i-Societal Stereotypes',
               'Explicit Marking of Sex', 'i-Explicit Marking of Sex']

label2id = {k: v for v, k in enumerate(temp_labels)}
id2label = {v: k for v, k in enumerate(temp_labels)}
label2id

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained("/content/gdrive/MyDrive/bias_detection_syetem/bias_detection_model_bert-base-uncased")

### Load test datasets

In [6]:
# test_custom_labeled = pd.read_csv('Data/test_custom_labeled.csv')
# test_hugging_face = pd.read_csv('Data/test_hugging_face.csv')
test_annotated = pd.read_csv('/content/gdrive/MyDrive/bias_detection_syetem/test_data.csv')

In [15]:
test_annotated.values[13]

array([106, 'O,O,O,O,O,O,Behavioural Stereotypes,O,O,O,O,O,O,O,O,O,O,O,O',
       'you will be a highly motivated, committed, fast learner looking to take the next step in your career.'],
      dtype=object)

In [ ]:
# test_custom_labeled = pd.read_csv('Data/test_custom_labeled.csv')
# test_hugging_face = pd.read_csv('Data/test_hugging_face.csv')
test_annotated = pd.read_csv('Data/test_data_annotated.csv')

In [16]:
len(test_annotated)

1009

In [ ]:
test_annotated.values[13]

array(['O,O,O,O,Societal Stereotypes,O,O,O,Behavioural Stereotypes,O,O,O,O,O,O,O',
       'The post involves supporting individuals to live as independently as possible in their own homes.'],
      dtype=object)

In [ ]:
len(test_annotated)

161

### Predict single sentence

In [25]:
sentence = test_annotated.sentence[13]
sentence = "We are committed to building better neighborhoods wherever we are, not only for our residents, but for the greater community"
sentence = "Ideal candidates will be caring, compassionate, adaptable, flexible, committed to high standards of care and possess excellent communication skills both verbal and written."
sentence = "the post involves supporting people to live as independently as possible in their own homes ."
#sentence = "A programmer must carry his laptop with him to work." 
#sentence = "A nurse should ensure that she  gets adequate rest"
#sentence = "He / She is suitable for this position."
#sentence = "Must be an extrovert who has innate quality of easily connecting with people!"
#sentence = "Are you the master of technology and passionate person we are looking for?"
inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)

outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

m = torch.nn.Softmax(dim=1)
probabilities = m(logits[0])
prediction_probability = torch.max(probabilities, axis=1)[0].tolist()
prediction_probability = [ '%.3f' % elem for elem in prediction_probability ]

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions, prediction_probability)) # list of tuples. Each tuple = (wordpiece, prediction)


word_level_predictions = []
word_level_probabilities = []
for pair in wp_preds:
    if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
        continue
    else:
        word_level_predictions.append(pair[1])
        word_level_probabilities.append(pair[2])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print()
print('PRED:' + str(word_level_predictions))
#print(word_level_probabilities)
#print('TRUE:' + str(test_annotated.word_labels[13].split(',')))

the post involves supporting people to live as independently as possible in their own homes .

PRED:['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Behavioural Stereotypes', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### Use softmax for probabilities

In [26]:
m = torch.nn.Softmax(dim=1)
probabilities = m(logits[0])
max_probability = torch.max(probabilities, axis=1)
boolean_mask = torch.as_tensor(list(map(bool,mask[0])))
filtered_probabilities = torch.masked_select(max_probability[0], boolean_mask).tolist()
filtered_probabilities = [ '%.2f' % elem for elem in filtered_probabilities ]

In [27]:
print(filtered_probabilities)

['1.00', '1.00', '1.00', '1.00', '1.00', '0.93', '1.00', '1.00', '1.00', '0.68', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']


In [28]:
test_annotated

,Unnamed: 0,word_labels,sentence
0,2,"O,O,O,O,O,O,O,O,Behavioural Stereotypes,O,O,O,...",The incumbentshould be a quick learner who is ...
1,17,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,Behavioural Stereo...",Having previously worked within a busy Contact...
2,18,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Behavioural ...",Applications are invited from employees who ar...
3,37,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","To be responsible, in conjunction with the hea..."
4,48,"O,O,O,O,O,O,O,O,O,O,O,O",A professional approach to hitting tough reven...
...,...,...,...
1004,4012,"O,O,O,O,O,O,O,O,O,O,O,O,Generic he,",The supervisor had a quarrel with the attendan...
1005,4016,"O,O,O,O,O,O,Generic he,O,O,O,O,",The mechanic hired the tailor because he could...
1006,4022,"O,O,O,O,O,O,O,O,O,O,O,Generic she,",The assistant helped the cook for preparing fo...
1007,4024,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Explicit M...",To take responsibility for the day to day runn...
